<a href="https://colab.research.google.com/github/Tender-sun/ML/blob/main/RNN%E5%AE%9E%E7%8E%B0_sr.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### 一、基于pytorch的实现

In [ ]:
import torch
import torch.nn as nn
import torchvision
from torchvision import datasets,transforms
from torch.autograd import Variable
from matplotlib import pyplot as plt
 
device = torch.device('cuda')
 
class RNN(nn.Module):
    def __init__(self):
        super().__init__()
        self.rnn = nn.RNN(
                input_size = 28,
                hidden_size = 1,
                num_layers = 1,
                batch_first = True,
        )
        self.Out2Class = nn.Linear(28,10)
    def forward(self, input):
        output,hn = self.rnn(input,None)
        # print('hn,shape:{}'.format(hn.shape))
        outreshape = output[:,:,0]
        # print(outreshape.shape)
        tmp = self.Out2Class(outreshape)
        # print(tmp.shape)
        return tmp
 
 
model = RNN()
model = model.to(device)
print(model)
 
 
 
model = model.train()
 
img_transform = transforms.Compose([transforms.ToTensor(),
                transforms.Normalize([0.5],[0.5])])
dataset_train = datasets.MNIST(root = './data',transform = img_transform,train = True,download = True)
dataset_test = datasets.MNIST(root = './data',transform = img_transform,train = False,download = True)
 
train_loader = torch.utils.data.DataLoader(dataset = dataset_train,batch_size=64,shuffle = True)
test_loader = torch.utils.data.DataLoader(dataset = dataset_test,batch_size=64,shuffle = False)
 
# images,label = next(iter(train_loader))
# print(images.shape)
# print(label.shape)
# images_example = torchvision.utils.make_grid(images)
# images_example = images_example.numpy().transpose(1,2,0)
# mean = [0.5,0.5,0.5]
# std = [0.5,0.5,0.5]
# images_example = images_example*std + mean
# plt.imshow(images_example)
# plt.show()
 
def Get_ACC():
    correct = 0
    total_num = len(dataset_test)
    for item in test_loader:
        batch_imgs,batch_labels = item
        batch_imgs = batch_imgs.squeeze(1)
        batch_imgs = Variable(batch_imgs)
        batch_imgs = batch_imgs.to(device)
        batch_labels = batch_labels.to(device)
        out = model(batch_imgs)
        _,pred = torch.max(out.data,1)
        correct += torch.sum(pred==batch_labels)
    correct = correct.data.item()
    acc = correct/total_num
    print('correct={},Test ACC:{:.5}'.format(correct,acc))
optimizer = torch.optim.Adam(model.parameters())
loss_f = nn.CrossEntropyLoss()
 
Get_ACC()
for epoch in range(100):
    print('epoch:{}'.format(epoch))
    cnt = 0
    for item in train_loader:
        batch_imgs ,batch_labels = item
        batch_imgs = batch_imgs.squeeze(1)
        batch_imgs,batch_labels = Variable(batch_imgs),Variable(batch_labels)
        batch_imgs = batch_imgs.to(device)
        batch_labels = batch_labels.to(device)
        out = model(batch_imgs)
        loss = loss_f(out,batch_labels)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        if(cnt%100==0):
            print_loss = loss.data.item()
            #print('epoch:{},cnt:{},loss:{}'.format(epoch,cnt,print_loss))
        cnt+=1
    Get_ACC()
 
 
torch.save(model,'model')

RNN(
  (rnn): RNN(28, 1, batch_first=True)
  (Out2Class): Linear(in_features=28, out_features=10, bias=True)
)
correct=1019,Test ACC:0.1019
epoch:0
correct=6671,Test ACC:0.6671
epoch:1
correct=6886,Test ACC:0.6886
epoch:2
correct=6986,Test ACC:0.6986
epoch:3
correct=7043,Test ACC:0.7043
epoch:4
correct=7100,Test ACC:0.71
epoch:5
correct=7171,Test ACC:0.7171
epoch:6
correct=7209,Test ACC:0.7209
epoch:7
correct=7244,Test ACC:0.7244
epoch:8
correct=7289,Test ACC:0.7289
epoch:9
correct=7304,Test ACC:0.7304
epoch:10
correct=7333,Test ACC:0.7333
epoch:11
correct=7312,Test ACC:0.7312
epoch:12
correct=7366,Test ACC:0.7366
epoch:13
correct=7360,Test ACC:0.736
epoch:14
correct=7362,Test ACC:0.7362
epoch:15
correct=7381,Test ACC:0.7381
epoch:16
correct=7378,Test ACC:0.7378
epoch:17
correct=7409,Test ACC:0.7409
epoch:18
correct=7394,Test ACC:0.7394
epoch:19
correct=7418,Test ACC:0.7418
epoch:20
correct=7401,Test ACC:0.7401
epoch:21
correct=7429,Test ACC:0.7429
epoch:22
correct=7414,Test ACC:0.7414

### 二、基于keras的实现

In [ ]:
import numpy as np
np.random.seed(1337)  # for reproducibility
 
from keras.datasets import mnist
from keras.utils import np_utils
from keras.models import Sequential
from keras.layers import SimpleRNN, Activation, Dense
from keras.optimizers import adam_v2
 
TIME_STEPS = 28     # same as the height of the image
INPUT_SIZE = 28     # same as the width of the image
BATCH_SIZE = 64
BATCH_INDEX = 0
OUTPUT_SIZE = 10
CELL_SIZE = 50
LR = 0.001
MAX_ITER=20001
 
 
# download the mnist to the path '~/.keras/datasets/' if it is the first time to be called
# X shape (60,000 28x28), y shape (10,000, )
(X_train, y_train), (X_test, y_test) = mnist.load_data()
 
# data pre-processing
X_train = X_train.reshape(-1, 28, 28) / 255.      # normalize
X_test = X_test.reshape(-1, 28, 28) / 255.        # normalize
y_train = np_utils.to_categorical(y_train, num_classes=10)
y_test = np_utils.to_categorical(y_test, num_classes=10)
 
# build RNN model
model = Sequential()
 
# RNN cell
model.add(SimpleRNN(
    # for batch_input_shape, if using tensorflow as the backend, we have to put None for the batch_size.
    # Otherwise, model.evaluate() will get error.
    batch_input_shape=(None, TIME_STEPS, INPUT_SIZE),       # Or: input_dim=INPUT_SIZE, input_length=TIME_STEPS,
    units=CELL_SIZE,
    unroll=True,
))
 
# output layer
model.add(Dense(OUTPUT_SIZE))
model.add(Activation('softmax'))
 
# optimizer
adam = adam_v2.Adam(LR)
model.compile(optimizer=adam,
              loss='categorical_crossentropy',
              metrics=['accuracy'])
 
# training
for step in range(MAX_ITER):
    # data shape = (batch_num, steps, inputs/outputs)
    X_batch = X_train[BATCH_INDEX: BATCH_INDEX+BATCH_SIZE, :, :]
    Y_batch = y_train[BATCH_INDEX: BATCH_INDEX+BATCH_SIZE, :]
    cost = model.train_on_batch(X_batch, Y_batch)
    BATCH_INDEX += BATCH_SIZE
    BATCH_INDEX = 0 if BATCH_INDEX >= X_train.shape[0] else BATCH_INDEX
 
    if step % 500 == 0:
        cost, accuracy = model.evaluate(X_test, y_test, batch_size=y_test.shape[0], verbose=False)
        print('iter = ',step,'test cost: ', cost, 'test accuracy: ', accuracy)

11501568/11490434 [==============================] - 0s 0us/step
iter =  0 test cost:  2.4051592350006104 test accuracy:  0.10400000214576721
iter =  500 test cost:  0.5619312524795532 test accuracy:  0.8371999859809875
iter =  1000 test cost:  0.4255611300468445 test accuracy:  0.8729000091552734
iter =  1500 test cost:  0.3316791355609894 test accuracy:  0.9056000113487244
iter =  2000 test cost:  0.2989089787006378 test accuracy:  0.9156000018119812
iter =  2500 test cost:  0.28790420293807983 test accuracy:  0.9168999791145325
iter =  3000 test cost:  0.27327030897140503 test accuracy:  0.9217000007629395
iter =  3500 test cost:  0.2863106429576874 test accuracy:  0.916700005531311
iter =  4000 test cost:  0.2580949366092682 test accuracy:  0.9258000254631042
iter =  4500 test cost:  0.2799329459667206 test accuracy:  0.9229999780654907
iter =  5000 test cost:  0.20786210894584656 test accuracy:  0.9423999786376953
iter =  5500 test cost:  0.21665668487548828 test accuracy:  0.9383